# Agenda
Analytical data modeling involves designing a data model that supports reporting and analytics. Analytics can be defined as the **the systematic computational analysis of data.** The chief aim of analytical data modeling is to interprete and communicate meaningful patterns extracted from a data set and apply them towards effective decision making. 

It is important for analyst to shift his point of view from **transactional operations on data (i.e. in OLTP)** to **analytical operations on data (i.e. in OLAP).** Analytics and Reporting are performed on data models (i.e. ORM, Cube), which are constructed on top of (mostly) transactional datasets. 

In this exercise, we demonstrate how to prepare data for both ORM and Cube modeling. We describe a few OLAP operations that can be implemented on a Cube model. The content for the remainder of this guide is as follows.

## Table of Contents
1. [Python Libraries](#libraries)  
    1.1 [Installing Python Libraries](#lib-install)  
    1.2 [Importing Python Libraries](#lib-import)  
2. [Preparing Data](#data-set)  
    2.1 [Loading Data into Database](#loading)  
    2.2 [ORM Modeling](#orm)  
3. [OLAP Cube Operations](#olap)  
    3.1 [Cube Modeling](#cube)  
    3.2 [Creating Workspace](#workspace)  
    3.3 [Data Aggregation](#aggregation)  
    3.4 [Drill Down](#drill-down)  
    3.5 [Slicing and Dicing](#slice-dice)  
4. [Exercise: Multidimensional Modeling on a UCI Data set](#exercise)  



# 1. Python Libraries <a name="libraries"></a>
## 1.1 Install Python libraries <a name="lib-install"></a>
This exercise will requires the following **Python** libraries:

<ul>
    <li><strong>cubes:</strong> a light-weight Python framework for OLAP</li>
    <li><strong>SQLAlchemy:</strong> a Python SQL Toolkit and Object Relational Mapper for ORM</li>
</ul>

In [1]:
# Installing Libraries (if not installed)
!pip3 install cubes
!pip3 install sqlalchemy==1.4.41

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached cubes-1.1-py3-none-any.whl
  Using cached expressions-0.2.3-py3-none-any.whl
  Using cached grako-3.99.9-py2.py3-none-any.whl (82 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils confi

## 1.2 Import libraries <a name="lib-import"></a>

In [1]:
# Importing Libraries

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session

from cubes.tutorial.sql import create_table_from_csv
from cubes.compat import ConfigParser
from cubes import Workspace, Cell, PointCut


# 2. Preparing Data <a name="data-set"></a>
In this section, we demostrate how to prepare data for ORM and Cube modeling. First, we create a SQLite database and populate it with an example dataset (IBRD Balance Sheet) taken from [The World Bank](https://finances.worldbank.org/Accounting-and-Control/IBRD-Balance-Sheet-FY2010/e8yz-96c6). This data is saved in [data.csv](data.csv), which will be the **data source** for this exercise.

Some parts of this exercise was adopted from:
1. [Cubes - OLAP Framework](https://pythonhosted.org/cubes/index.html)
2. [DataBrewery/cubes](https://github.com/DataBrewery/cubes/tree/master/examples/hello_world)

## 2.1 Loading Data into Database <a name="loading"></a>

In [2]:
# Prepare SQL data in memory

FACT_TABLE = "irbd_balance"

engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "data.csv",
                      table_name=FACT_TABLE,
                      fields=[
                            ("category", "string"),
                            ("category_label", "string"),
                            ("subcategory", "string"),
                            ("subcategory_label", "string"),
                            ("line_item", "string"),
                            ("year", "integer"),
                            ("amount", "integer")],
                      create_id=True
                  )

## 2.2 ORM Modeling <a name="orm"></a>
In this section, we demonstrate how to use the **SQLAlchemy ORM Python API** to construct and execute ORM queries (*i.e SELECT, INSERT, UPDATE, DELETE*).

Some parts of this exercise was adopted from [SQLAlchemy Documentation](https://docs.sqlalchemy.org/en/14/orm/tutorial.html).


In [10]:
# Implementing a SQL ORM (Object Relational Model)

DBSession = scoped_session(sessionmaker(bind=engine))
session = DBSession()

result = session.execute("SELECT * FROM irbd_balance LIMIT 25")
#result = session.execute("SELECT * FROM irbd_balance WHERE category='e'")


for row in result.fetchall():
    # print(row)
    print(str(row.category_label)+' ' +str(row.year) + ' ' +str(row.line_item))
    # print("\n")
    #print("%-20s%10s%10d%10d" % (row.category_label,
    #                          row.subcategory,
    #                          #row.line_item,
    #                          row.year,
    #                          row.amount,
    #                          ))

Assets 2010 Unrestricted currencies
Assets 2009 Unrestricted currencies
Assets 2010 Currencies subject to restriction
Assets 2009 Currencies subject to restriction
Assets 2010 Trading
Assets 2009 Trading
Assets 2010 Securities purchased under resale agreements
Assets 2009 Securities purchased under resale agreements
Assets 2010 Nonnegotiable, nonintrest-bearing demand obligations on account of subscribed capital
Assets 2009 Nonnegotiable, nonintrest-bearing demand obligations on account of subscribed capital
Assets 2010 Investments
Assets 2009 Investments
Assets 2010 Client operations
Assets 2009 Client operations
Assets 2010 Borrowings
Assets 2009 Borrowings
Assets 2010 Other
Assets 2009 Other
Assets 2010 Receivables to maintain value of currency holdings on account of subscribed capital
Assets 2009 Receivables to maintain value of currency holdings on account of subscribed capital
Assets 2010 Receivables from investment securities traded
Assets 2009 Receivables from investment securi

# 3. OLAP Cube Operations <a name="olap"></a>
In this section, we apply a **Cube model** on the data prepared in the section above and we use the Cube model to perform a few OLAP operations (*i.e. drill-down, slicing, dicing*)

## 3.1 Cube Modeling <a name="cube"></a>
Cube modeling allows us to descibe a **logical model** from an analyst's perspective (i.e. data how they are being measured, aggregated and reported). This model is independent of physical implementation of data. For example, the logical model enable users to:

1. see the data from the business perspective.
2. hide physical structure of the data (“how application’s use it”).
3. specify OLAP dimension hierarchies of attributes.
4. provide more descriptive attribute labels for display in the applications or reports.
5. transparent localization of metadata and data.

The logical model is described using **model metadata dictionary**. This dictionary is composed of:
1. logical part,
2. physical part and,
3. any other additional information of the logical model.

![Logical Model](./img/logical_model.png)

The logical model is implemented using *JSON*. In this exercise, we use an example model [model.json](model.json)

### Reference (For more Info)
* [Logical Model and Metadata](https://pythonhosted.org/cubes/model.html)




## 3.2 Creating Workspace <a name="workspace"></a>
Everything in Cubes happens in an **analytical workspace**. A **workspace** contains cubes and maintains connections to the data stores (with cube data). The workspace properties are specified in a configuration file [slicer.ini](slicer.ini) (default name). In the configuration file, we specify at least the following items:

* data store (i.e. in this exercise - it is the SQLite)
* Cube model (created in the section above)
* etc ...

After creating a **workspace**, we need a **AggregationBrowser -** an object that allows us to perform multi-dimensional data (or Cube) queries and operations such as:

* aggregation, 
* slicing, 
* dicing, 
* browsing




In [11]:
# Create a workspace

conf = ConfigParser()
conf.read('slicer.ini')
workspace = Workspace(config=conf)

In [12]:
# Get a browser

browser = workspace.browser('irbd_balance')

## 3.3 Data Aggregation  <a name="aggregation"></a>
**Data aggregation** is a query in a multi-dimensional data space and it aims to compute/consolidate data for one or more dimensions to a measure (i.e. sum).

<img src="./img/aggregate.png" alt="drawing" width="400"/>

Image Source: [Slideshare](https://www.slideshare.net/pommurajopt/data-pre-processing-31974618)


In [13]:
# Play with aggregates

result = browser.aggregate()
result.summary

{'amount_sum': 1116860, 'double_amount_sum': 2233720, 'record_count': 62}

## 3.4 Drill Down <a name="drill-down"></a>
A way of viewing related items of a Dimension as defined in a Hierarchy by expanding Members to access a more detailed data range; put simply, this means displaying Child Members associated with a specific Parent Member or Aggregate Member within a defined Hierarchy.

<img src="./img/drill_down.png" alt="drawing" width="600"/>

Image source: [Tutorialspoint](https://www.tutorialspoint.com/dwh/dwh_olap.htm)

In [20]:
# Drill-down by item dimension
result = browser.aggregate(drilldown=["item"])
# print(result)
print("Drill-down by Item")
for row in result.table_rows("item"):
    # print(str(row))
    # print("\n")
    print("%-20s%10d%10d%10d" % ( row.label,
                              row.record["record_count"],
                              row.record["amount_sum"],
                              row.record["double_amount_sum"],
                              ))
print("\n ---- \n")
  
# Drill-down by year dimension
result = browser.aggregate(drilldown=["year"])
print("Drill-down by Year")
for row in result.table_rows("year"):
    print("%-20s%10d%10d%10d" % (row.label,
                              row.record["record_count"],
                              row.record["amount_sum"],
                              row.record["double_amount_sum"],
                              ))

Drill-down by Item
TableRow(key='a', label='Assets', path=['a'], is_base=False, record={'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 558430, 'double_amount_sum': 1116860, 'record_count': 32})


Assets                      32    558430   1116860
TableRow(key='e', label='Equity', path=['e'], is_base=False, record={'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 77592, 'double_amount_sum': 155184, 'record_count': 8})


Equity                       8     77592    155184
TableRow(key='l', label='Liabilities', path=['l'], is_base=False, record={'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 480838, 'double_amount_sum': 961676, 'record_count': 22})


Liabilities                 22    480838    961676

 ---- 

Drill-down by Year
2009                        31    550840   1101680
2010                        31    566020   1132040


## 3.5 Slicing and Dicing <a name="slice-dice"></a>

### 3.5.1 Slicing
A Slice represents two-dimensional view of an OLAP Cube that arranges data in a grid, similar to a spreadsheet; a Slice functions much like a report or a query in an RDBMS. In this exercise, a slice is implemented using the **Cut object**.

<img src="./img/slicing.png" alt="drawing" width="500"/>


### 3.5.2 Dicing
The dice operation produces a subcube by allowing the analyst to pick specific values of multiple dimensions. In this exercise, a dice is implemented using the **Cell object**.

<img src="./img/dicing.png" alt="drawing" width="500"/>


Image sources: [Tutorialspoint](https://www.tutorialspoint.com/dwh/dwh_olap.htm)

### Reference (More Info)
* [Slicing and Dicing](https://pythonhosted.org/cubes/slicing_and_dicing.html)

In [9]:
# Slice -> Cut
# ------------

# Slice (or cut) for the Year 2010
    
cut_1 = PointCut("year", [2010])


# Dice (or cut) for Item 'e' for the year 2009
cut_2 = [ 
    PointCut("item", ["e"]), 
    PointCut("year", [2009])
]

# Dice -> Cell
cell_1 = Cell(browser.cube, cuts = [cut_1])
cell_2 = Cell(browser.cube, cuts = cut_2)


print("Slice: Year 2010")
result = browser.aggregate(cell_1)
print(result.summary)

print("\n\n ----------- \n\n")


print("Dice: Item 'e' for Year 2009")
result = browser.aggregate(cell_2)
print(result.summary)


# print(result)
#for row in result.table_rows("year"):
#    print("%-20s%10d%10d%10d" % ( row.label,
#                              row.record["record_count"],
#                              row.record["amount_sum"],
#                              row.record["double_amount_sum"],
#                              ))
    
# cube = workspace.cube("irbd_balance")
# result = browser.aggregate(cell, drilldown=["item"])
# for row in result.table_rows("item"):
#    print("%-20s%10d%10d%10d" % ( row.label,
#                              row.record["record_count"],
#                              row.record["amount_sum"],
#                              row.record["double_amount_sum"],
#                              ))

Slice: Year 2010
{'amount_sum': 566020, 'double_amount_sum': 1132040, 'record_count': 31}


 ----------- 


Dice: Item 'e' for Year 2009
{'amount_sum': 40037, 'double_amount_sum': 80074, 'record_count': 4}


# 4. Exercise: Multidimensional Modeling on a UCI Data set <a name="exercise"></a>
The [PRSA_Data.csv](PRSA_Data.csv) is retrieved from [UCI Data Repository](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data). The data (**Beijing Multi-Site Air-Quality Data Data Set**): is hourly data set considers 6 main air pollutants and 6 relevant meteorological variables at multiple sites in Beijing. Using this data set:

1. Prepare the data (load into an SQLite database)
2. design a Cube model with dimensions: **Wind direction (wd)**, **Year** and **Date (aggregates Year, Month and Day)**.
3. use the Cube to perform OLAP operations (i.e. Drill-down, Slicing, Dicing)